# 코드의 방향성
### 멜론 장르음악 스테디셀러 차트
    - 장르별로 약 300개씩
    - 한 페이지에 50곡이 있고 6페이지씩 정보를 크롤링하려 함
    - 6페이지가 안 되는 장르가 있음. -> 그냥 그 장르 곡 최대수만큼만 크롤링
    - 각 노래마다 제목, 가수, 가사, 장르를 뽑아서 DataFrame에 저장
    - 국내뿐만 아니라 해외 곡들의 장르도 크롤링 하려 함.
        - 우리가 만든 모델을 사용하는 사용자가 영어를 썼을 때 값을 내보내주기 위해.

#### https://www.melon.com/genre/song_list.htm?gnrCode=GN0100&steadyYn=Y 기반이 되는 사이트
#### https://www.melon.com/song/detail.htm?songId=3414749

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 구글 드라이브 마운트. 드라이브 내에 존재하는 파일이 런타임 재실행 때마다 사라지지 않고 남아있도록 하는 코드. 
# dataframe을 csv파일로 저장하고 현재 이 ipynb 파일이 있는 드라이브 폴더에 그 csv파일을 넣어놓으면 나중에 다시 켤 때마다 계속 새로 추가하지 않아도 됨.

Mounted at /content/drive


In [ ]:
# 셀레니움, 크롬드라이버 설치. 아마 한번만 실행하면 될듯. 런타임 재실행해도 남아있을 걸? 아마두
!apt-get update > /dev/null 2>&1
!pip install selenium > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1

In [ ]:
#각종 모듈 import
import time # 새로운 페이지로 셀레니음 웹드라이버가 넘어갈 때마다 잠시 멈춰주는 역할을 위해 불러옴. 
# 멈춰주지 않으면 페이지가 다 로딩되기도 전에 다음 코드를 실행해버리는 불상사가 생김. 그걸 막아준다.
from selenium import webdriver # 웹드라이버. 코랩이라 우리가 직접 볼 순 없지만 얘가 돌아다니면서 일을 해줌
from selenium.webdriver.common.keys import Keys # 이 파일에선 아직 딱히 쓰지 않은듯
from selenium.webdriver.common.by import By # element를 찾을 때 쓰이는 parameter를 불러오는 역할
import pandas as pd 
from bs4 import BeautifulSoup # html 요소에 접근해서 정보를 빼내올 때 씀
from tqdm import tqdm
import pickle
import numpy as np

In [ ]:
import warnings # 사실 잘 모르지만 선생님이 하셨던거 암튼 가져옴
warnings.filterwarnings('ignore')

In [ ]:
#크롬 드라이버의 각종 옵션들 생성 및 적용.
options = webdriver.ChromeOptions()
options.add_argument('--headless') # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--single-process')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36') # 나는 기계가 아니고 사람이에요
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
base_url = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN' # 뒤에 0100 ~ 1400를 붙여주려 함. 100씩 증가하며 장르가 바뀜. 국내 8종, 해외 6종
std = '&steadyYn=Y' # 스테디셀러 페이지로 가는 parameter. 
info_base_url = 'https://www.melon.com/song/detail.htm?songId=' # 각 노래의 정보를 담은 페이지. 뒤에 각 노래의 id값을 넣어주려 한다.
genre = {1:'balad', 2:'dance', 3:'hiphop', 4:'rnb', 5:'inde', 6:'rock', 7:'trot', 
         8:'folk', 9:'pop', 10:'f_rock', 11:'electronica', 12:'f_hiphop', 13:'f_rnb', 14:'country'} # 각 장르에 번호를 붙임

너무 오래 걸리니까 5번 나눠서 하자

In [ ]:
song_info = [] # 노래 정보 담을 리스트

# 장르별로 for문 돌기
for i in range(1,4):
    genre_info = genre[i] # 어떤 장르인지
    if i < 10: # url에 0100 0200처럼 숫자를 붙일 건데 1000부터는 걍 i에 100만 곱하면 되므로 if문으로 조건을 갈라줌
        url = base_url + '0' + str(i * 100) + std
    else:
        url = base_url + str(i * 100) + std

    # 각 장르별 페이지를 움직이려 함
    for page_num in range(6):
        driver.get(url)
        if page_num != 0: # 1번 페이지일 때는 페이지를 넘길 필요가 없으므로 그 외의 페이지들만 해당하도록 if문 작성
            try:
                driver.find_element(By.XPATH, f'//*[@id="pageObjNavgation"]/div/span/a[{page_num}]').click() 
            # 6번 페이지보다 적은 장르가 몇 있다. 그 때 에러가 뜨므로 try except문으로 탈출, 다음 장르로 넘어가게 됨
            except:
                break

        # 집 네트워크 상태가 영판 안좋으면 sleep 수를 조금씩 늘려도 좋을듯. 대신 시간은 그만큼 배로 늘어나긴 할 것이다.
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser') # 페이지가 넘어갔을 때 
        inputs = soup.select('tbody .input_check') # 노래 id들을 담은 요소가 담긴 리스트를 만듦

        for inp in tqdm(inputs):
            id = inp['value'] # 각 노래 id값 꺼내기
            info_url = info_base_url + id  # 노래 정보가 있는 페이지 url
            driver.get(info_url) # 거기로 이동
            time.sleep(1)
            try:
                lyric = driver.find_element(By.CSS_SELECTOR, "#d_video_summary").text 
            # 19금 노래는 로그인을 해야만 가사가 나옴. 그냥 꺼내려 하면 에러가 뜨기 때문에 일단 건너뛰려고 예외처리
            except:
                continue
            lyric = lyric.replace('\n\n','\n').split('\n') 
            # 가끔 가사 사이에 개행문자 두개가 있는 경우가 있다. 그걸 하나로 바꿔주고 개행문자를 기준으로 문장을 나눠 리스트로 저장

            title = driver.find_element(By.CSS_SELECTOR,'.song_name').text.strip() # 노래 제목
            artist = driver.find_element(By.CSS_SELECTOR, '.artist_name').text.strip() # 가수명
            song_info.append([title, artist, lyric, genre_info]) # 노래 정보를 담아준다.

100%|██████████| 50/50 [02:30<00:00,  3.01s/it]


In [ ]:
# ck_df = pd.DataFrame(song_info, columns = ['title','artist','lyric','genre'])
# len(np.unique(ck_df.title))

848

In [ ]:
# with open('song_info.p', 'wb') as f:
#     pickle.dump(song_info, f)
# 0816 완료

In [ ]:
with open('/content/drive/MyDrive/프로젝트/song_info.p', 'rb') as f:
    song_info = pickle.load(f)
len(song_info)

867

In [ ]:
# 장르별로 for문 돌기
for i in range(4,7):
    genre_info = genre[i] # 어떤 장르인지
    if i < 10: # url에 0100 0200처럼 숫자를 붙일 건데 1000부터는 걍 i에 100만 곱하면 되므로 if문으로 조건을 갈라줌
        url = base_url + '0' + str(i * 100) + std
    else:
        url = base_url + str(i * 100) + std

    # 각 장르별 페이지를 움직이려 함
    for page_num in range(6):
        driver.get(url)
        if page_num != 0: # 1번 페이지일 때는 페이지를 넘길 필요가 없으므로 그 외의 페이지들만 해당하도록 if문 작성
            try:
                driver.find_element(By.XPATH, f'//*[@id="pageObjNavgation"]/div/span/a[{page_num}]').click() 
            # 6번 페이지보다 적은 장르가 몇 있다. 그 때 에러가 뜨므로 try except문으로 탈출, 다음 장르로 넘어가게 됨
            except:
                break

        # 집 네트워크 상태가 영판 안좋으면 sleep 수를 조금씩 늘려도 좋을듯. 대신 시간은 그만큼 배로 늘어나긴 할 것이다.
        time.sleep(1.5)
        soup = BeautifulSoup(driver.page_source, 'html.parser') # 페이지가 넘어갔을 때 
        inputs = soup.select('tbody .input_check') # 노래 id들을 담은 요소가 담긴 리스트를 만듦

        for inp in tqdm(inputs):
            id = inp['value'] # 각 노래 id값 꺼내기
            info_url = info_base_url + id  # 노래 정보가 있는 페이지 url
            driver.get(info_url) # 거기로 이동
            time.sleep(1.5)
            try:
                lyric = driver.find_element(By.CSS_SELECTOR, "#d_video_summary").text 
            # 19금 노래는 로그인을 해야만 가사가 나옴. 그냥 꺼내려 하면 에러가 뜨기 때문에 일단 건너뛰려고 예외처리
            except:
                continue
            lyric = lyric.replace('\n\n','\n').split('\n') 
            # 가끔 가사 사이에 개행문자 두개가 있는 경우가 있다. 그걸 하나로 바꿔주고 개행문자를 기준으로 문장을 나눠 리스트로 저장

            title = driver.find_element(By.CSS_SELECTOR,'.song_name').text.strip() # 노래 제목
            artist = driver.find_element(By.CSS_SELECTOR, '.artist_name').text.strip() # 가수명
            song_info.append([title, artist, lyric, genre_info]) # 노래 정보를 담아준다.

100%|██████████| 50/50 [02:56<00:00,  3.52s/it]


In [ ]:
ck_df = pd.DataFrame(song_info, columns = ['title','artist','lyric','genre'])
len(np.unique(ck_df.title)), len(song_info)
# 이상하게 저장된거 같으면 위에서 파일을 다시 읽고 다시 실행!

(1632, 1756)

In [ ]:
ck_df

In [ ]:
with open('/content/drive/MyDrive/프로젝트/song_info.p', 'wb') as f:
    pickle.dump(song_info, f)

In [ ]:
with open('/content/drive/MyDrive/프로젝트/song_info.p', 'rb') as f:
    song_info = pickle.load(f)
len(song_info)

1756

In [ ]:
# 장르별로 for문 돌기
for i in range(7,10):
    genre_info = genre[i] # 어떤 장르인지
    if i < 10: # url에 0100 0200처럼 숫자를 붙일 건데 1000부터는 걍 i에 100만 곱하면 되므로 if문으로 조건을 갈라줌
        url = base_url + '0' + str(i * 100) + std
    else:
        url = base_url + str(i * 100) + std

    # 각 장르별 페이지를 움직이려 함
    for page_num in range(6):
        driver.get(url)
        if page_num != 0: # 1번 페이지일 때는 페이지를 넘길 필요가 없으므로 그 외의 페이지들만 해당하도록 if문 작성
            try:
                driver.find_element(By.XPATH, f'//*[@id="pageObjNavgation"]/div/span/a[{page_num}]').click() 
            # 6번 페이지보다 적은 장르가 몇 있다. 그 때 에러가 뜨므로 try except문으로 탈출, 다음 장르로 넘어가게 됨
            except:
                break

        # 집 네트워크 상태가 영판 안좋으면 sleep 수를 조금씩 늘려도 좋을듯. 대신 시간은 그만큼 배로 늘어나긴 할 것이다.
        time.sleep(1.5)
        soup = BeautifulSoup(driver.page_source, 'html.parser') # 페이지가 넘어갔을 때 
        inputs = soup.select('tbody .input_check') # 노래 id들을 담은 요소가 담긴 리스트를 만듦

        for inp in tqdm(inputs):
            id = inp['value'] # 각 노래 id값 꺼내기
            info_url = info_base_url + id  # 노래 정보가 있는 페이지 url
            driver.get(info_url) # 거기로 이동
            time.sleep(1.5)
            try:
                lyric = driver.find_element(By.CSS_SELECTOR, "#d_video_summary").text 
            # 19금 노래는 로그인을 해야만 가사가 나옴. 그냥 꺼내려 하면 에러가 뜨기 때문에 일단 건너뛰려고 예외처리
            except:
                continue
            lyric = lyric.replace('\n\n','\n').split('\n') 
            # 가끔 가사 사이에 개행문자 두개가 있는 경우가 있다. 그걸 하나로 바꿔주고 개행문자를 기준으로 문장을 나눠 리스트로 저장

            title = driver.find_element(By.CSS_SELECTOR,'.song_name').text.strip() # 노래 제목
            artist = driver.find_element(By.CSS_SELECTOR, '.artist_name').text.strip() # 가수명
            song_info.append([title, artist, lyric, genre_info]) # 노래 정보를 담아준다.

100%|██████████| 17/17 [00:53<00:00,  3.13s/it]


In [ ]:
ck_df = pd.DataFrame(song_info, columns = ['title','artist','lyric','genre'])
len(np.unique(ck_df.title)), len(song_info)
# 이상하게 저장된거 같으면 위에서 파일을 다시 읽고 다시 실행!

(2258, 2534)

In [ ]:
ck_df.tail(300)

In [ ]:
with open('/content/drive/MyDrive/프로젝트/song_info.p', 'wb') as f:
    pickle.dump(song_info, f)

In [ ]:
with open('/content/drive/MyDrive/프로젝트/song_info.p', 'rb') as f:
    song_info = pickle.load(f)
len(song_info)

2534

In [ ]:
# 장르별로 for문 돌기
for i in range(10,13):
    genre_info = genre[i] # 어떤 장르인지
    if i < 10: # url에 0100 0200처럼 숫자를 붙일 건데 1000부터는 걍 i에 100만 곱하면 되므로 if문으로 조건을 갈라줌
        url = base_url + '0' + str(i * 100) + std
    else:
        url = base_url + str(i * 100) + std

    # 각 장르별 페이지를 움직이려 함
    for page_num in range(6):
        driver.get(url)
        if page_num != 0: # 1번 페이지일 때는 페이지를 넘길 필요가 없으므로 그 외의 페이지들만 해당하도록 if문 작성
            try:
                driver.find_element(By.XPATH, f'//*[@id="pageObjNavgation"]/div/span/a[{page_num}]').click() 
            # 6번 페이지보다 적은 장르가 몇 있다. 그 때 에러가 뜨므로 try except문으로 탈출, 다음 장르로 넘어가게 됨
            except:
                break

        # 집 네트워크 상태가 영판 안좋으면 sleep 수를 조금씩 늘려도 좋을듯. 대신 시간은 그만큼 배로 늘어나긴 할 것이다.
        time.sleep(1.5)
        soup = BeautifulSoup(driver.page_source, 'html.parser') # 페이지가 넘어갔을 때 
        inputs = soup.select('tbody .input_check') # 노래 id들을 담은 요소가 담긴 리스트를 만듦

        for inp in tqdm(inputs):
            id = inp['value'] # 각 노래 id값 꺼내기
            info_url = info_base_url + id  # 노래 정보가 있는 페이지 url
            driver.get(info_url) # 거기로 이동
            time.sleep(1.5)
            try:
                lyric = driver.find_element(By.CSS_SELECTOR, "#d_video_summary").text 
            # 19금 노래는 로그인을 해야만 가사가 나옴. 그냥 꺼내려 하면 에러가 뜨기 때문에 일단 건너뛰려고 예외처리
            except:
                continue
            lyric = lyric.replace('\n\n','\n').split('\n') 
            # 가끔 가사 사이에 개행문자 두개가 있는 경우가 있다. 그걸 하나로 바꿔주고 개행문자를 기준으로 문장을 나눠 리스트로 저장

            title = driver.find_element(By.CSS_SELECTOR,'.song_name').text.strip() # 노래 제목
            artist = driver.find_element(By.CSS_SELECTOR, '.artist_name').text.strip() # 가수명
            song_info.append([title, artist, lyric, genre_info]) # 노래 정보를 담아준다.

100%|██████████| 32/32 [01:49<00:00,  3.41s/it]


In [ ]:
ck_df = pd.DataFrame(song_info, columns = ['title','artist','lyric','genre'])
len(np.unique(ck_df.title)), len(song_info)
# 이상하게 저장된거 같으면 위에서 파일을 다시 읽고 다시 실행!

(2753, 3076)

In [ ]:
ck_df.tail(100)

In [ ]:
with open('/content/drive/MyDrive/프로젝트/song_info.p', 'wb') as f:
    pickle.dump(song_info, f)

In [ ]:
with open('/content/drive/MyDrive/프로젝트/song_info.p', 'rb') as f:
    song_info = pickle.load(f)
len(song_info)

3076

In [ ]:
# 장르별로 for문 돌기
for i in range(13,15):
    genre_info = genre[i] # 어떤 장르인지
    if i < 10: # url에 0100 0200처럼 숫자를 붙일 건데 1000부터는 걍 i에 100만 곱하면 되므로 if문으로 조건을 갈라줌
        url = base_url + '0' + str(i * 100) + std
    else:
        url = base_url + str(i * 100) + std

    # 각 장르별 페이지를 움직이려 함
    for page_num in range(6):
        driver.get(url)
        if page_num != 0: # 1번 페이지일 때는 페이지를 넘길 필요가 없으므로 그 외의 페이지들만 해당하도록 if문 작성
            try:
                driver.find_element(By.XPATH, f'//*[@id="pageObjNavgation"]/div/span/a[{page_num}]').click() 
            # 6번 페이지보다 적은 장르가 몇 있다. 그 때 에러가 뜨므로 try except문으로 탈출, 다음 장르로 넘어가게 됨
            except:
                break

        # 집 네트워크 상태가 영판 안좋으면 sleep 수를 조금씩 늘려도 좋을듯. 대신 시간은 그만큼 배로 늘어나긴 할 것이다.
        time.sleep(1.5)
        soup = BeautifulSoup(driver.page_source, 'html.parser') # 페이지가 넘어갔을 때 
        inputs = soup.select('tbody .input_check') # 노래 id들을 담은 요소가 담긴 리스트를 만듦

        for inp in tqdm(inputs):
            id = inp['value'] # 각 노래 id값 꺼내기
            info_url = info_base_url + id  # 노래 정보가 있는 페이지 url
            driver.get(info_url) # 거기로 이동
            time.sleep(1.5)
            try:
                lyric = driver.find_element(By.CSS_SELECTOR, "#d_video_summary").text 
            # 19금 노래는 로그인을 해야만 가사가 나옴. 그냥 꺼내려 하면 에러가 뜨기 때문에 일단 건너뛰려고 예외처리
            except:
                continue
            lyric = lyric.replace('\n\n','\n').split('\n') 
            # 가끔 가사 사이에 개행문자 두개가 있는 경우가 있다. 그걸 하나로 바꿔주고 개행문자를 기준으로 문장을 나눠 리스트로 저장

            title = driver.find_element(By.CSS_SELECTOR,'.song_name').text.strip() # 노래 제목
            artist = driver.find_element(By.CSS_SELECTOR, '.artist_name').text.strip() # 가수명
            song_info.append([title, artist, lyric, genre_info]) # 노래 정보를 담아준다.

df = pd.DataFrame(song_info, columns = ['title','artist','lyric','genre'])
df

100%|██████████| 50/50 [02:32<00:00,  3.04s/it]


,title,artist,lyric,genre
0,내 손을 잡아,아이유,"[느낌이 오잖아, 떨리고 있잖아, 언제까지 눈치만 볼 거니, 네 맘을 말해봐, 딴청...",balad
1,너의 모든 순간 (Original),성시경,"[이윽고 내가 한눈에, 너를 알아봤을 때, 모든 건 분명 달라지고 있었어, 내 세상...",balad
2,Timeless,SG 워너비,"[어쩜 살아가다 보면, 한 번은 날 찾을지 몰라, 난 그 기대 하나로, 오늘도 힘겹...",balad
3,바람이 불었으면 좋겠어,길구봉구,"[웃는 모습이 너무 아름다운, 그런 너를 기억하면서, 괜찮아 넌 잘할거라 말하던, ...",balad
4,봄 사랑 벚꽃 말고,HIGH4 (하이포),"[길었던 겨우내 줄곧 품이 좀 남는, 밤색 코트, 그 속에 나를 쏙 감추고 걸음을,...",balad
...,...,...,...,...
3604,The City,Ed Sheeran,"[This city never sleeps, I hear the people wal...",country
3605,500 Miles (Bonus Track),Innocence Mission,"[If you miss the train I'm on, You will know t...",country
3606,My Sweet Lady,John Denver,"[My Sweet Lady, , Lady, are you crying,, do th...",country
3607,Skyrocket,Suehee,"[If we could sky rocket away, Up-up and away i...",country


In [ ]:
df.to_csv('노래_정보.csv', index=False) # 파일을 불러올 때는 이거 쓰는게 편함
df.to_csv('노래_정보_euckr.csv', index=False, encoding='euc-kr') # 엑셀 파일 열어서 볼 때 한글 안 깨지게 인코딩 바꿔준 것

UnicodeEncodeError: ignored

In [ ]:
len(song_info)

3609

In [ ]:
with open('/content/drive/MyDrive/프로젝트/song_info.p', 'wb') as f:
    pickle.dump(song_info, f)